In [18]:
import pandas as pd
import numpy as np

In [19]:
df = pd.read_csv(r"/workspaces/Supply-Chain-Management/Data/supplier_contracts_dataset.csv")
df = df.replace({np.nan: None})

# # Convert only the relevant text-based fields to string
text_fields = ['contracttype', 'suppliername', 'risklevel', 'complianceissues', 'keyterms', 'negotiationrecommendation','qualityissues', 'supplychaindisruptions',
       'increasedcosts', 'complianceandlegalrisks', 'missedopportunities',
       'damagedrelationships']

# Ensure the specified text fields are strings
for field in text_fields:
    df[field] = df[field].astype(str)

**MINSEARCH**

In [20]:
# df.columns = df.columns.str.lower()
# df.to_csv(r"/workspaces/Supply-Chain-Management/Data/supplier_contracts_dataset.csv",index=False)

In [21]:
documents = df.to_dict(orient='records')

Get top 10 list of high risk level contracts

In [22]:
import minsearch

# Create an index
index = minsearch.Index(
    text_fields=text_fields,
    keyword_fields=[]
)

# Fit the index with the documents
index.fit(documents)

# Example search query
query = "high risk level"
results = index.search(query, num_results=10)

# Print results
for result in results:
    print(result)

{'contractid': 309, 'datesigned': '5/19/2023', 'contracttype': 'Service Agreement', 'suppliername': 'Supplier_33', 'risklevel': 'High', 'complianceissues': 'Non-Compliance with Standards', 'keyterms': 'Delivery within 60 days', 'pastperformancescore': 9, 'negotiationrecommendation': 'Adjust delivery schedules', 'qualityissues': 'Yes', 'supplychaindisruptions': 'Yes', 'increasedcosts': 'Yes', 'complianceandlegalrisks': 'High', 'missedopportunities': 'Yes', 'damagedrelationships': 'No', 'qualitymetrics': 10, 'deliverymetrics': 8, 'costmetrics': 4, 'relationshipmetrics': 8}
{'contractid': 329, 'datesigned': '12/21/2023', 'contracttype': 'Distribution Agreement', 'suppliername': 'Supplier_85', 'risklevel': 'High', 'complianceissues': 'Non-Compliance with Standards', 'keyterms': 'Quality standards as per ISO 9001', 'pastperformancescore': 5, 'negotiationrecommendation': 'Seek alternative suppliers', 'qualityissues': 'Yes', 'supplychaindisruptions': 'No', 'increasedcosts': 'No', 'compliancea

Get the Contract types that has high risk and their count

In [23]:
import minsearch
from collections import Counter
# Create an index
index = minsearch.Index(
    text_fields=text_fields,
    keyword_fields=['risklevel']
)

# Fit the index with the documents
index.fit(documents)

# Perform the search for high-risk level contracts
filter_dict = {'risklevel': 'High'}
results = index.search(query='high risk level', filter_dict=filter_dict, num_results=len(documents))

# Extract and print contract types with high risk level
high_risk_contract_types = [result['contracttype'] for result in results]

# Count the occurrences of each contract type
contract_type_counts = Counter(high_risk_contract_types)

# Print the count of each contract type
print("Count of each contract type with high risk level:")
for contract_type, count in contract_type_counts.items():
    print(f"{contract_type}: {count}")

Count of each contract type with high risk level:
Supply Agreement: 156
Procurement Contract: 176
Distribution Agreement: 165
Manufacturing Agreement: 183
Service Agreement: 170


**GROQ API**

In [24]:
# from openai import OpenAI

# client = OpenAI()

In [25]:
import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [26]:
df.columns

Index(['contractid', 'datesigned', 'contracttype', 'suppliername', 'risklevel',
       'complianceissues', 'keyterms', 'pastperformancescore',
       'negotiationrecommendation', 'qualityissues', 'supplychaindisruptions',
       'increasedcosts', 'complianceandlegalrisks', 'missedopportunities',
       'damagedrelationships', 'qualitymetrics', 'deliverymetrics',
       'costmetrics', 'relationshipmetrics'],
      dtype='object')

In [27]:

# Function to call the LLM (Groq API)
# def llm(prompt, model='gpt-4o-mini'):
#     # Assuming `client` is the Groq API client instance
#     response = client.chat.completions.create(
#         model=model,
#         messages=[{"role": "user", "content": prompt}]
#     )
    
#     return response.choices[0].message.content



In [28]:
df.columns


Index(['contractid', 'datesigned', 'contracttype', 'suppliername', 'risklevel',
       'complianceissues', 'keyterms', 'pastperformancescore',
       'negotiationrecommendation', 'qualityissues', 'supplychaindisruptions',
       'increasedcosts', 'complianceandlegalrisks', 'missedopportunities',
       'damagedrelationships', 'qualitymetrics', 'deliverymetrics',
       'costmetrics', 'relationshipmetrics'],
      dtype='object')

In [34]:

# Function to perform the search on supplier contracts based on query

def search(query, filter_dict=None, max_results=10):
    # Filter the DataFrame based on risk level (if provided)
    if filter_dict:
        filtered_df = df[df['risklevel'] == filter_dict.get('risklevel', '')]
    else:
        filtered_df = df
    # Convert the filtered data to a list of dictionaries and limit the number of results
    results = filtered_df.to_dict(orient='records')[:max_results]
    return results

# Function to build a clearer prompt for Groq API
def build_clear_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context += (
            f"- **ContractType**: {doc['contracttype']}\n"
            f"  **SupplierName**: {doc['suppliername']}\n"
            f"  **RiskLevel**: {doc['risklevel']}\n"
            f"  **ComplianceIssues**: {doc['complianceissues']}\n"
            f"  **KeyTerms**: {doc['keyterms']}\n"
            f"  **NegotiationRecommendation**: {doc['negotiationrecommendation']}\n"
            f"  **QualityIssues**: {doc['qualityissues']}\n"
            f"  **SupplyChainDisruptions**: {doc['supplychaindisruptions']}\n"
            f"  **IncreasedCosts**: {doc['increasedcosts']}\n"
            f"  **ComplianceAndLegalRisks**: {doc['complianceandlegalrisks']}\n"
            f"  **MissedOpportunities**: {doc['missedopportunities']}\n"
            f"  **DamagedRelationships**: {doc['damagedrelationships']}\n"
            f"  **QualityMetrics**: {doc['qualitymetrics']}\n"
            f"  **DeliveryMetrics**: {doc['deliverymetrics']}\n"
            f"  **CostMetrics**: {doc['costmetrics']}\n"
            f"  **RelationshipMetrics**: {doc['relationshipmetrics']}\n\n"
        )
    
    prompt = (
        f"QUESTION: {query}\n\n"
        f"CONTEXT:\n{context}"
    )
    
    return prompt

# Function to call the LLM (Groq API)
# def llm(prompt, model='gpt-4o-mini'):
#     # Assuming client is the Groq API client instance
#     response = client.chat.completions.create(
#         model=model,
#         messages=[{"role": "user", "content": prompt}]
#     )
    
#     return response.choices[0].message.content

def llm(prompt, model='Llama3-groq-70b-8192-tool-use-preview'):
    # Assuming client is the Groq API client instance
    response = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
    model=model
    )
    
    return response.choices[0].message.content


# Function to perform the full RAG (Retrieve and Generate) process
def rag(query, model='Llama3-groq-70b-8192-tool-use-preview'):
    # Search for high-risk contracts (you can modify filter_dict based on needs)
    search_results = search(query, filter_dict={'risklevel': 'High'})
    
    # Build the prompt using the search results
    prompt = build_clear_prompt(query, search_results)
    
    # Get the LLM response based on the prompt
    answer = llm(prompt, model=model)
    
    return answer

# Example usage
question = "Give contract types , qualitymetrics ,SupplyChainDisruptions and their negotiation recommendations for high-risk contracts"
answer = rag(question)
print(answer)

Based on the provided context, here are the contract types, quality metrics, supply chain disruptions, and negotiation recommendations for high-risk contracts:

1. **ContractType**: Distribution Agreement
   **SupplierName**: Supplier_60
   **RiskLevel**: High
   **QualityMetrics**: 3
   **SupplyChainDisruptions**: No
   **NegotiationRecommendation**: Include compliance monitoring

2. **ContractType**: Service Agreement
   **SupplierName**: Supplier_67
   **RiskLevel**: High
   **QualityMetrics**: 5
   **SupplyChainDisruptions**: Yes
   **NegotiationRecommendation**: Include compliance monitoring

3. **ContractType**: Supply Agreement
   **SupplierName**: Supplier_33
   **RiskLevel**: High
   **QualityMetrics**: 3
   **SupplyChainDisruptions**: Yes
   **NegotiationRecommendation**: Include penalty clauses for late delivery

4. **ContractType**: Service Agreement
   **SupplierName**: Supplier_24
   **RiskLevel**: High
   **QualityMetrics**: 9
   **SupplyChainDisruptions**: Yes
   **Nego

In [37]:
print("\n## Risk-Based Queries")

question = "Which suppliers have the most non-compliance issues, regardless of risk level?"
answer = rag(question)
print(answer)

print("\n##Compliance & Legal Queries:")


question = "Identify contracts where compliance monitoring is needed."
answer = rag(question)
print(answer)

print("\n##Cost and Financial Metrics:")
question = "List contracts that provide cost savings but involve high risk."
answer = rag(question)
print(answer)

print("\n##Contractual Terms and Recommendations:")
question = "Which contracts have penalty clauses for late delivery, and how effective have they been?"
answer = rag(question)
print(answer)

print("\n##Supplier Relationship Queries:")
question = "What are the relationship metrics for high-performing suppliers?"
answer = rag(question)
print(answer)


print("\n##Opportunity and Innovation Queries:")
question = "Identify suppliers that provide innovative opportunities despite low past performance scores."
answer = rag(question)
print(answer)


print("\n#Custom Queries")

question = "Show me all contracts with a combination of poor delivery metrics, compliance risks, and high past performance scores."
answer = rag(question)
print(answer)

print("\n")
question ="What are the patterns between non-compliance and increased costs in supplier contracts?"
answer = rag(question)
print(answer)






## Risk-Based Queries
Supplier_60, Supplier_67, Supplier_33, Supplier_24, Supplier_16, Supplier_29, Supplier_62, Supplier_20, Supplier_28, and Supplier_79 all have high-risk levels and various compliance issues. However, Supplier_67 has the most non-compliance issues, including late delivery and compliance with standards.

##Compliance & Legal Queries:
Supplier_60, Supplier_67, Supplier_33, Supplier_24, Supplier_16, Supplier_29, Supplier_62, Supplier_20, Supplier_28, and Supplier_79 all require compliance monitoring.

##Cost and Financial Metrics:
Supplier_60, Supplier_67, Supplier_33, Supplier_24, Supplier_16, Supplier_29, Supplier_62, Supplier_20, Supplier_28, and Supplier_79

##Contractual Terms and Recommendations:
The contracts with penalty clauses for late delivery are:
1. **ContractType**: Supply Agreement
   **SupplierName**: Supplier_33
   **NegotiationRecommendation**: Include penalty clauses for late delivery

2. **ContractType**: Manufacturing Agreement
   **SupplierName**

**Retrieval Evaluation**